# Assignment 4: Image and Video Alignment

## General Information
**Submmision:** in pairs.

## General Instructions

The work you submit **must** run in the environment `wis-cv` which we provided.

Do not change function headers. Where `**kwargs` is provided, use it to add any input you wish, and add it to the documentation.

You may use the following [tutorial](http://cs231n.github.io/python-numpy-tutorial/) for Python.

## Introduction

In this exercise, you are asked to implement the 2D parametric estimation and alignment algorithm that was presented in class. The implementation should be for the case of a global 2D translation (i.e., a global image shift). Note that this is a special case of a homography, where
$$ H = 
\begin{bmatrix}
    1 & 0 & \Delta x \\
    0 & 1 & \Delta y \\
    0 & 0 & 1
\end{bmatrix}
$$

The exercise has two main parts:

* **Images:** Align an image with respect to a reference image.
* **Videos:** Align a video with respect to a specific frame in the video. Use this to create a clean version of the frame, and to remove fast moving objects from the video.

In [1]:
from collections import namedtuple                       # for testing

import numpy as np                                       # numpy
import skimage.transform                                 # images
from skimage.color import rgb2gray
import matplotlib.pyplot as plt                          # plotting

import ipywidgets as widgets

# ex4 imports
from pyramids import pyramid_kernel, gaussian_reduce     # pyramids
from pyramids import DEFAULT_DEPTH, DEFAULT_KERNEL       # default params
from utils import imread, imwrite, imshow                # images
from utils import vread, vwrite, vshow                   # video

# your plot outputs will appear and be stored in the notebook
%matplotlib inline

## 1 Images (50 points)

In this part you will align an image with respect to a reference image, and test your results. You will implement the following methods: 

* **Align images**
    * `find_shift()`
    * `align_image()`
* **Test align images**
    * `root_mean_square_error()`
    * `peak_signal_noise_ratio()`
    * `error_map()`
    * `test_align_image()`

### 1.1 Core

In [2]:
# PROVIDED FUNCTIONS
def warp(image, shift):
    """Applies a shift on an image.
    Fills the undefined areas with NaNs.
    
    Args:
        image (np.ndarray): image to be warped.
        shift (np.ndarray): shift to warp with.
        
    Returns:
        np.ndarray: warped image.
    """
    inv_mat = np.eye(3)
    inv_mat[0:2, 2] = -shift
    return skimage.transform.warp(image, inv_mat, cval=np.nan)

#### [Q1.1.1] Implement `find_shift()` (25 points)

Find the shift between two images. Use the _Hierarchical Lucas & Kanade Method_:
1. Start the process at the smallest pyramid level and iterate `iters` times per level.
2. Use the shift from one level as an initialization for the next level. When doing this, remember to scale it accordingly.

Technical details:
* In order to perform the warping between images, you can use the provided `warp()` method.
* You can use the provided pyramid methods imported at the beginning.
* Use [`np.gradient`](https://docs.scipy.org/doc/numpy/reference/generated/numpy.gradient.html) for taking the image derivatives.

In [3]:
def find_shift(image, reference, **kwargs):
    """Finds the best shift between `image` and `reference`.
    
    It should satisfy (up to small epsilon error):
        warp(image, find_shift(image, reference)) == reference
    
    Args:
        image (np.ndarray): the image to warp.
        reference (np.ndarray): the image to warp to.
        iters (int): number of iterations per pyramid-level (default: 5).
        
    Optional Args:
        iters (int): number of iterations per pyramid-level (default: 5).
        depth (int): depth of the pyramid (default: 5).
        kernel (np.ndarray): kernel for the pyramid (default: with `a=0.375`).
   
    Your Args:
        *TODO*: describe here your additional **kwargs variables.
    
    Returns:
        results (dict): dictionary of results, with the following keys:
                            results['shift']: an np.ndarray describing the shift in the form [dx, dy].
                        may contain additional data (under other keys) for testing, e.g. for `test_image_align()`.       
    """
    
    # parse given arguments
    iters = kwargs.pop('iters', 5)
    depth = kwargs.pop('depth', DEFAULT_DEPTH)
    kernel = kwargs.pop('kernel', DEFAULT_KERNEL)
    
    # YOUR CODE HERE
    results = {'shift_log': []}
    reference_pyramid = gaussian_reduce(reference, kernel=kernel, depth=depth)
    image_pyramid = gaussian_reduce(image, kernel=kernel, depth=depth)
    
    total_shift = np.zeros(2)
    for layer in reversed(range(depth)):
        ref = reference_pyramid[layer]
        image = image_pyramid[layer]
        
        Iy, Ix, _ = np.gradient(image)

        M = np.array([[(Ix**2).sum(), (Ix * Iy).sum()],
                      [(Ix * Iy).sum(), (Iy**2).sum()]])
        invM = np.linalg.pinv(M)

        total_shift *= 2
        for i in range(iters):
            results['shift_log'].append(-total_shift * 2**layer)

            current_ref = warp(ref, total_shift)  # Shift J to I (as was asked in class)
            It = image - current_ref

            b = - np.array([np.nansum(Ix * It), np.nansum(Iy * It)])            
            
            shift = invM @ b
            total_shift += shift
            

    results['shift_log'].append(-total_shift)
    results['shift'] = -total_shift  # The required shift is from I to J so a minus sign is needed
    
    return results

In [4]:
### HIDDEN AUTOMATIC TESTS #1


In [5]:
### HIDDEN AUTOMATIC TESTS #2


In [6]:
### HIDDEN AUTOMATIC TESTS #3


#### [Q1.1.2] Implement `align_image()` (5 points)

Align `image` to `reference`. Use the `find_shift()` method implemented above.

In [7]:
def align_image(image, reference, **kwargs):
    """Finds a shift between `image` and `reference` and applies it on `image`.
    
    Args:
        image (np.ndarray): the image to warp.
        reference (np.ndarray): the image to warp to.
        
    Optional Args:
        iters (int): number of iterations per pyramid-level (default: 5).
        depth (int): depth of the pyramid (default: 5).
        kernel (np.ndarray): kernel for the pyramid (default: with `a=0.375`).
    
    Your Args:
        *TODO*: describe here your additional **kwargs variables.
    
    Returns:
        results (dict): dictionary of results, with the following keys:
                            results['shift']: an np.ndarray describing the shift in the form [dx, dy].
                            results['aligned_image']: `image` aligned to match `reference`.
                        may contain additional data (under other keys) for testing, e.g. for `test_image_align()`.        
    """

    # YOUR CODE HERE
    results = find_shift(image, reference, **kwargs)
    results['aligned_image'] = warp(image, results['shift'])
    
    return results

In [8]:
### HIDDEN AUTOMATIC TESTS

### 1.2 Evaluation Utilities (20 points)

#### [Q1.2.1] Implement the three following methods (10 points)

These will be used to evaluate and visualize the operation of `align_image()`.

In [9]:
def root_mean_square_error(image, reference):
    """Finds the root-mean-square error between `image` and `reference`.
    
    .. [1] https://en.wikipedia.org/wiki/Root-mean-square_deviation
    
    Args:
        image (np.ndarray): image #1.
        reference (np.ndarray): image #2.
    
    Returns:
        rmse (float): root-mean-square error between `image` and `reference`.
    """
    # YOUR CODE HERE
    return float(np.sqrt(np.nanmean((reference-image)**2)))

In [10]:
def peak_signal_noise_ratio(image, reference, bounds=(0, 1)):
    """Finds the peak signal-to-noise ratio between `image` and `reference`.
    
    .. [1] https://en.wikipedia.org/wiki/Peak_signal-to-noise_ratio
    
    Args:
        image (np.ndarray): image #1.
        reference (np.ndarray): image #2.
        bounds (Tuple[float, float]): the minimal and maximal allowed pixel values.
    
    Returns:
        psnr (float): peak signal-to-noise in dB.
    """

    # YOUR CODE HERE
    MIN, MAX = bounds

    image = np.clip(image, MIN, MAX)
    reference = np.clip(reference, MIN, MAX)
    rmse = root_mean_square_error(image, reference)
    
    return 20 * np.log10((MAX-MIN)/rmse)

In [11]:
def error_map(image, reference, bounds=(0, 1), only_magnitude=False, zero_nans=False):
    """Creates a map of the difference between `image` and `reference`.
        
    Args:
        image (np.ndarray): image #1.
        reference (np.ndarray): image #2.
        bounds (Tuple[float, float]): the minimal and maximal allowed pixel values.
        only_magnitude (bool): whether `diff` should contain only the magnitude of `image - reference` (default: False).
        zero_nans (bool): replace NaNs by zeros (default: False).
        
    Returns:
        diff (np.ndarray): map of the differences between `image` and `reference`.
        bounds (Tuple[float, float]): the valid bounds of diff.
    """
    # YOUR CODE HERE
    MIN, MAX = bounds
    image = np.clip(image, MIN, MAX)
    reference = np.clip(reference, MIN, MAX)
    
    diff = image - reference
    diff_bounds = MIN-MAX, MAX-MIN

    if only_magnitude:
        diff = np.linalg.norm(diff, 2)
        
    if zero_nans:
        diff = np.nan_to_num(diff)
    
    
    return diff, diff_bounds

In [12]:
### HIDDEN AUTOMATIC TESTS

In [13]:
### HIDDEN AUTOMATIC TESTS

In [14]:
### HIDDEN AUTOMATIC TESTS

#### [Q1.2.2] Implement `test_align_image()` (10 points)

Implement `test_align_image()`, which will visualize the shifting and warping of a given image-refernce pair by `align_image()`. 
`test_align_image()` should create the following plots:

* The original and the reference image, side by side
* The original and aligned image, side by side
* The difference images (original vs. reference, and original vs. aligned), side by side, with relevant RMSE values in the title.
* A graph of the translation calculated in each iteration in each pyramid level. Note that the translation value needs to be scaled according to the appropriate pyramid level.

You may use the functions `imshow_hbox()` and `imshow_tabs()` provided above.

Your output should be similar to the following example:
![example of test_align_image](docs/test_align_image.png "example of test_align_image")


In [15]:
## PROVIDED METOHDS
def imshow_tabs(images, tab_names, titles=None, kwargs=None):
    """Normalize `images`, and show them in tabs with the given tab names.
    
    Args:
        images (List[np.ndarray]): list of images to show.
        tab_names (List[str]): list of image tab names.
        bounds (List[Tuple[int, int]]): the minimal and maximal allowed pixel values for each image (if needed)      
    """
    assert len(images) == len(tab_names)
    if not isinstance(titles, (list, tuple)):
        titles = [titles] * len(images)
    assert len(images) == len(titles)
    if kwargs is None:
        kwargs = {}
    n = len(images)
    if isinstance(kwargs, dict):
        kwargs = [kwargs for _ in range(n)]
    outputs = [widgets.Output() for _ in range(n)]
    boxes = widgets.Tab(children=outputs)
    for i, (output, image, tab_name, title, kw) in enumerate(zip(outputs, images, tab_names, titles, kwargs)):
        boxes.set_title(i, tab_name)
        with output:
            imshow(image, title=title, **kw)
    
    display(boxes)
    
def imshow_hbox(images, titles=None, kwargs=None):
    """Normalize `images`, and show them side-by-side.

    Args:
        images (List[np.ndarray]): list of images to show.
        tab_names (List[str]): list of image tab names.
        bounds (List[Tuple[int, int]]): the minimal and maximal allowed pixel values for each image (if needed)    
    """
    if kwargs is None:
        kwargs = {}
    if not isinstance(titles, (list, tuple)):
        titles = [titles] * len(images)
    assert len(images) == len(titles)

    n = len(images)
    if isinstance(kwargs, dict):
        kwargs = [kwargs for _ in range(n)]
    outputs = [widgets.Output() for _ in range(n)]
    boxes = widgets.HBox(children=outputs)
    for i, (output, image, title, kw) in enumerate(zip(outputs, images, titles, kwargs)):
        with output:
            imshow(image, title=title, **kw)
    
    display(boxes)

In [16]:
def test_align_image(path_image, path_reference, name, **kwargs):
    """Creates visualization of the shifting and warping of a given image-refernce pair.
    
    Args:
        path_image (str): image's path.
        path_reference (str): reference image's path.
        name (str): name for the given image-reference instance.
        
    Optional Args:
        iters (int): number of iterations per pyramid-level (default: 5).
        depth (int): depth of the pyramid (default: 5).
        kernel (np.ndarray): kernel for the pyramid (default: with `a=0.375`).
   
    Your Args:
        *TODO*: describe here your additional **kwargs variables.        
    """
    image = imread(path_image)
    reference = imread(path_reference)
    
    results = align_image(image, reference, *kwargs)
    aligned_image = results['aligned_image']
    
    original_error_map, original_bounds = error_map(image, reference)
    aligned_error_map, aligned_bounds = error_map(aligned_image, reference)
    
    original_rmse = root_mean_square_error(image, reference)
    aligned_rmse = root_mean_square_error(aligned_image, reference)
    
    imshow_hbox([image, reference], ['Image', 'Reference'])
    imshow_hbox([image, aligned_image], ['Image', 'Aligned Image'])
    imshow_hbox([original_error_map, aligned_error_map],
                ['Difference Before Alignment: RMSE {}'.format(original_rmse),
                 'Difference After Alignment: RMSE {}'.format(aligned_rmse)],
               kwargs=[{"bounds": original_bounds}, {"bounds": aligned_bounds}])
    
    shift_log = np.array(results['shift_log'])

    plt.plot(np.arange(0, len(shift_log)), shift_log[:, 0], '.-', label='dx') # Plot X
    plt.plot(np.arange(0, len(shift_log)), shift_log[:, 1], '.-', label='dy') # Plot Y
    
    plt.xlabel('Iteration (N)')
    plt.ylabel('Translation (pixels)')
    plt.legend()
    plt.title('Translation vs. Iterations')
    
    plt.show()


In [17]:
ImagePair = namedtuple("ImagePair", "name path_image path_reference")

examples = [
    ImagePair("Airport", "data/pairs/airport/01.jpg", "data/pairs/airport/02.jpg"),
    ImagePair("Parachute", "data/pairs/parachute/01.jpg", "data/pairs/parachute/02.jpg")
]

canvas_list = widgets.Accordion(children=[widgets.Output() for _ in range(len(examples))])
for i, (example, canvas) in enumerate(zip(examples, canvas_list.children)):
    canvas_list.set_title(i, example.name)
    with canvas:
        test_align_image(example.path_image, example.path_reference, example.name)
display(canvas_list)

Accordion(children=(Output(), Output()), _titles={'0': 'Airport', '1': 'Parachute'})

## 2 Videos (50 points)

In this part you will use the tool of image alignment to process videos. You will reduce noise of a video frame using information from other frames, and you will remove fast moving objects from a video.
You will implement the following methods:
* `align_video()`
* `reduce_noise()`
* `remove_dynamics()`

### 2.1 Core (20 points)

#### [Q2.1.1] Implement `align_video()` (15 points)

In this part, you need to align a video according to a reference frame.

In [18]:
def align_video(video, ref_idx, **kwargs):
    """Finds the shift between each frame in `video` to frame number `ref_idx`, and aligns all the frames to the reference frame.
        
    Args:
        video (np.ndarray): the video to align.
        ref_idx (int): index of the reference frame.
        
    Optional Args:
        iters (int): number of iterations per pyramid-level (default: 5).
        depth (int): depth of the pyramid (default: 5).
        kernel (np.ndarray): kernel for the pyramid (default: with `a=0.375`).
   
    Your Args:
        *TODO*: describe here your additional **kwargs variables.
    
    Returns:
        results (dict): dictionary of results, with the following keys:       
                            results['shift_list']: list of shifts between frames and the reference frame.
                            results['aligned_video']: `video` aligned w.r.t the reference frame.
                        may contain additional data (under other keys).
    """

# #   NAIVE SOLUTION:
#     results = {}
#     reference_frame = video[ref_idx,:,:,:]
    
#     results['aligned_video'] = np.zeros_like(video)
#     results['shift_list'] = []
#     for i, frame in enumerate(video):
#         res = align_image(frame,reference_frame) 
#         results['shift_list'].append(res['shift'])
#         results['aligned_video'][i,:,:,:] = res['aligned_image']
        
#     return results


    results = {}
    shifts = []
    results['aligned_video'] = np.zeros_like(video)
    results['shift_list'] = []
    
    for i in range(video.shape[0]-1):
        res = find_shift(video[i,:,:,:],video[i+1,:,:,:],**kwargs) 
        shifts.append(res['shift'])
    
    shifts_np = np.stack(shifts)
    
    # backwards loop from ref to 0 frame
    for i in range(0, ref_idx):
        results['shift_list'].append(np.sum(shifts_np[i:ref_idx,:],axis=0))

   
    # forwards loop from ref to end frame
    for i in range(ref_idx, video.shape[0]):
        results['shift_list'].append(-np.sum(shifts_np[ref_idx:i,:],axis=0))

    # alinging video frames with respect to summed shifts
    for i in range(video.shape[0]):
        results['aligned_video'][i,:,:,:] = warp(video[i,:,:,:], results['shift_list'][i])
    return results


    

In [19]:
video, meta = vread("data/video_with_michal.avi")
results = align_video(video, ref_idx=video.shape[0]//2)

vshow(video, title="Original Video", meta=meta)
vshow(results['aligned_video'], title="Aligned Video", fname="results/align_video/aligned_video.mp4", meta=meta)

C:\Users\guylu\anaconda3\envs\wis-cv\lib\site-packages\skimage\transform\_warps.py:655: RuntimeWarning: invalid value encountered in less_equal
  (min_val <= cval <= max_val))


Output()

Output()

In [20]:
### HIDDEN AUTOMATIC TESTS #1

In [21]:
### HIDDEN AUTOMATIC TESTS #2

#### [Q2.1.2] Theoretical Question (5 points)
Image alignment doesn't always work well when the shift is too big. How can you use additional information from the video to improve the accuracy of such "far" alignments?
Make sure to implement this improvement in `align_video()`. You may check it by looking at the alignment of the first/last frames.

A naive implementation of the algorithm would be to align each frame of the video to the reference frame independently.
This solution is indeed capable of aligning the video, yet it struggles when it comes to frames that are far away.
To remedy this we can introduce additional information from the rest of the frames in the video - take into account all intermidiate transformations from each frame to the reference frame and compose them (sum them up).
This way we get the total transformation from each frame to the reference frame with which can warp the frames to obtain the aligned video.

### 2.2 Noise Reduction (15 points)

#### [Q2.2.1] Implement `reduce_noise()` (10 points)

In this section we provide you with a noisy movie. The movie is of a static scene and is corrupted with noise.
When having multiple repetitions of the same pixel in different frame, we can remove the noise by taking the median of this pixel across all frames.

Technical details:
* read about [`np.nanmedian()`](https://docs.scipy.org/doc/numpy/reference/generated/numpy.nanmedian.html).

In [22]:
def reduce_noise(video, ref_idx, **kwargs):
    """Generates a clean version of a frame in a video, using information from other frames.
    
    Args:
        video (np.ndarray): the video to align.
        ref_idx (int): index of the reference frame.

    Your Args:
        *TODO*: describe here your additional **kwargs variables.
    
    Returns:
        np.ndarray: cleaned image.
    """
    # YOUR CODE HERE
    results = align_video(video, ref_idx, **kwargs)
    aligned_video = results['aligned_video']
    median_frame = np.nanmedian(aligned_video,axis=0)
    return median_frame

In [23]:
### HIDDEN AUTOMATIC TESTS

In [24]:
noisy_video, meta = vread("data/noisy_video.avi")

noisy_scene = noisy_video[noisy_video.shape[0]//2]
clean_scene = reduce_noise(noisy_video, ref_idx=noisy_video.shape[0]//2)

vshow(noisy_video, title="Noisy Video", meta=meta)
imshow(noisy_scene, title="Noisy Image")
imshow(clean_scene, title="Clean Image", fname="results/noise_reduction/clean_scene.png")
imshow(*error_map(clean_scene, noisy_scene), title="Noise", fname="results/noise_reduction/noise.png")

Output()

Output()

Output()

Output()

#### [Q2.2.2] Theoretical question (5 points)
Why do you think you were asked to use median rather than mean for the noise reduction?

The median is defined to be the value such that half of the data points are larger than it and half are less. 
The median is therefore susciptible only to the amount of the outliers in the sample (as oposed to their values), which is definitively small, therefore it should not dramatically affect the median value. 
The averge, however is susciptible to the outliers' values, which can really affect our results, due to extreme values (out of distribution).

In our case we don't want noisy outliers to affect our result, and so we need to come up with with a measure that is robust to outliers - hence the use of the median as opposed to the mean.

### 2.3 Remove Dynamics (15 points)

#### [Q2.3.1] Implement `remove_dynamics()` (10 points)

In this section we provide you with a movie which includes two motions: (i) camera pan; and (ii) a walking professor.
Applying alignment to the entire frame will lock on to the dominant motion (in this case the camera motion). Taking the median of each pixel across all frames would eliminate the walking professor from the entire movie.

In [25]:
def remove_dynamics(video, ref_idx, **kwargs):
    """Generates a clean version of a video, removing fast moving objects (and also noise).
    
    Args:
        video (np.ndarray): the video to align.
        ref_idx (int): index of the reference frame.

    Your Args:
        *TODO*: describe here your additional **kwargs variables.
    
    Returns:
        np.ndarray: cleaned video.
    """
    
    # YOUR CODE HERE
    removed_dyn_video = np.zeros_like(video)
    for i in range(video.shape[0]):
        removed_dyn_video[i:,:,:] = reduce_noise(video, i, **kwargs)
        
    return removed_dyn_video

In [26]:
### HIDDEN AUTOMATIC TESTS

In [27]:
video_with_michal, meta = vread("data/video_with_michal.avi")
video_without_michal = remove_dynamics(video_with_michal, ref_idx=video_with_michal.shape[0]//2)

vshow(video_with_michal, title="With Michal", meta=meta)
vshow(video_without_michal, title="Without Michal", fname="results/remove_dynamics/video_without_michal.mp4", meta=meta)

Output()

Output()

#### [Q2.3.2] Theoretical Question
Do you observe any artifacts in the output video (_without Michal_)? What is the cause of these artifacts? Can they be fixed?

Indeed we do see some artifacts. When the camera shifts its view to the left, we sudenlly see a vertical line separating the left side of the room (where the shelf is) from the right (where the cubicle is). We conjecture that the different exposure of the camera between the frames is to blame - since we noticed that the contrast changes throughout the video, resulting in change of intesities. 

To amend this, we suggest using pyramid blending technique, as was implemented in EX2. We would use every consecutive pair of frames and use the shifts between them to produce a separating mask (NaN in one image implies the pixes belonging to the other), and apply the pyramid blending algorithm consecutively.